In [1]:
# Dependencies
import json
import os
import pandas as pd

In [2]:
#open json
object_path = 'sample_data.json'
with open(object_path) as jsonfile:
       data_json = json.load(jsonfile)
df_from_json = pd.DataFrame(data_json)

In [3]:
df_from_json

,NCTId,ArmGroupInterventionName,ArmGroupType,BaselineMeasureDispersionType,BaselineMeasureParamType,CollaboratorClass,ConditionAncestorId,ConditionBrowseBranchAbbrev,ConditionBrowseLeafId,ConditionMeshId,...,InterventionType,IsFDARegulatedDevice,IsFDARegulatedDrug,LeadSponsorClass,OverallStatus,PatientRegistry,Phase,SamplingMethod,StdAge,StudyType
0,NCT04371159,Device: Velieve U.S.,Experimental,0,0,0,"[D000007239, D000014570]","[BC01, All, BXS]","[M8866, M4951, M15885, M15902]",D000014552,...,Device,Yes,No,INDUSTRY,Completed,0,Not Applicable,0,"[Adult, Older Adult]",Interventional
1,NCT04474496,Other: Assessing impact of COVID19,0,0,0,OTHER,0,"[BC18, BC19, All]","[M5698, M5702]",0,...,Other,No,No,OTHER,Enrolling by invitation,No,0,Non-Probability Sample,"[Adult, Older Adult]",Observational
2,NCT02801266,Behavioral: Evidence informed birth control co...,"[Experimental, No Intervention]",0,0,OTHER,0,0,0,0,...,Behavioral,0,0,OTHER,Completed,0,Not Applicable,0,Adult,Interventional
3,NCT01599390,Other: Data collection,0,0,0,INDUSTRY,"[D000009976, D000012327, D000014777, D00001214...","[BC02, BC08, All, BC01]","[M8878, M8866, M4951, M11485, M16105, M13732, ...",D000007251,...,Other,0,0,INDUSTRY,Completed,0,0,Non-Probability Sample,"[Child, Adult, Older Adult]",Observational
4,NCT00312702,[Biological: Falciparum Malaria Protein 11 wit...,"[Experimental, Experimental]",Standard Deviation,"[Mean, Count of Participants, Count of Partici...","[INDUSTRY, OTHER, FED]","[D000011528, D000010272]","[BC03, All, BC01, Rare]","[M17716, M9863, M8866, M4951, M12971, M11768, ...","[D000008288, D000016778]",...,Biological,0,0,FED,Completed,0,"[Phase 1, Phase 2]",0,Adult,Interventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NCT03768310,"[Biological: CD19.CAR-multiVST for Group A, Bi...","[Experimental, Experimental]",0,0,"[OTHER, OTHER]","[D000009370, D000009369, D000008232, D00000820...","[BC04, All, BC15, BC20, Rare]","[M9528, M9803, M26172, M9534, M10898, M9808, M...",D000054198,...,"[Biological, Biological]",No,Yes,OTHER,Not yet recruiting,0,Phase 1,0,"[Child, Adult, Older Adult]",Interventional
9996,NCT03768206,"[Behavioral: Standard Physical Therapy, Behavi...","[Experimental, Active Comparator]",0,0,OTHER,0,0,0,0,...,"[Behavioral, Behavioral, Behavioral]",No,No,OTHER,Completed,0,Not Applicable,0,"[Adult, Older Adult]",Interventional
9997,NCT03767738,Drug: Intravitreal Aflibercept Injection (IAI),Experimental,0,0,0,D000002318,"[BC14, All]",M15983,D000014652,...,Drug,No,Yes,INDUSTRY,Completed,0,Phase 4,0,"[Adult, Older Adult]",Interventional
9998,NCT03767595,Device: ProACT Adjustable Continence Therapy f...,Experimental,0,0,0,"[D000014555, D000014570, D000059411, D000020924]","[BXS, BC23, All, BXM]","[M15882, M6519, M15883, M15888, M15902, M28051...","[D000014549, D000014550]",...,Device,Yes,No,INDUSTRY,Recruiting,0,Not Applicable,0,"[Adult, Older Adult]",Interventional


In [4]:
# explodeTable function - to open a column from the list and create a new table from it
def explodeTable(df_name, column_name):
    tb = df_name[['NCTId', column_name]].copy()
    explode_tb = tb.explode(column_name)
    explode_tb = explode_tb.replace(0, "-")
    explode_tb = explode_tb.sort_values(by=[column_name], ascending=False)
    fr = len(explode_tb) - explode_tb[column_name].str.contains('-', na=False).sum()
    print('feature_row', fr)
    feature_tb = explode_tb.head(fr)
    #.rename(columns={column_name:'features'})

    return feature_tb

In [5]:
# extract "OverallStatus" value 
status_tb = df_from_json[['NCTId', 'OverallStatus']].copy()
status_tb = status_tb.explode('OverallStatus')
status_tb

,NCTId,OverallStatus
0,NCT04371159,Completed
1,NCT04474496,Enrolling by invitation
2,NCT02801266,Completed
3,NCT01599390,Completed
4,NCT00312702,Completed
...,...,...
9995,NCT03768310,Not yet recruiting
9996,NCT03768206,Completed
9997,NCT03767738,Completed
9998,NCT03767595,Recruiting


In [6]:
# extract "Phase" value
phase_tb = df_from_json[['NCTId', 'Phase']].copy()
phase_tb = phase_tb.explode('Phase')
phase_tb = phase_tb.replace({'Not Applicable':'n/a',0:'n/a','Early Phase 1': 'Phase_1', 'Phase 1': 'Phase_1', 'Phase 2': 'Phase_2', 'Phase 3': 'Phase_3', 'Phase 4': 'Phase_4'})
phase_tb

,NCTId,Phase
0,NCT04371159,n/a
1,NCT04474496,n/a
2,NCT02801266,n/a
3,NCT01599390,n/a
4,NCT00312702,Phase_1
...,...,...
9995,NCT03768310,Phase_1
9996,NCT03768206,n/a
9997,NCT03767738,Phase_4
9998,NCT03767595,n/a


In [7]:
# create table to split the Phases
phase_crosstb = pd.crosstab(index=phase_tb['NCTId'], columns = phase_tb['Phase']).reset_index()
phase_crosstb.sort_values('NCTId', inplace=True)
phase_crosstb

Phase,NCTId,Phase_1,Phase_2,Phase_3,Phase_4,n/a
0,NCT00000152,0,0,1,0,0
1,NCT00000157,0,0,1,0,0
2,NCT00000172,0,0,1,0,0
3,NCT00000173,0,0,1,0,0
4,NCT00000174,0,0,1,0,0
...,...,...,...,...,...,...
9995,NCT04608188,0,0,0,0,1
9996,NCT04608773,0,0,0,0,1
9997,NCT04609085,0,0,0,0,1
9998,NCT04609514,0,0,0,0,1


In [8]:
# create a new table with "Status"
df = df_from_json.copy()
df= df.drop(columns=['OverallStatus', 'Phase'])
df.insert(1,'Status',status_tb['OverallStatus'])
df.sort_values('NCTId', inplace=True)
df.reset_index(drop=True, inplace=True)
df

,NCTId,Status,ArmGroupInterventionName,ArmGroupType,BaselineMeasureDispersionType,BaselineMeasureParamType,CollaboratorClass,ConditionAncestorId,ConditionBrowseBranchAbbrev,ConditionBrowseLeafId,...,InterventionBrowseLeafId,InterventionMeshId,InterventionType,IsFDARegulatedDevice,IsFDARegulatedDrug,LeadSponsorClass,PatientRegistry,SamplingMethod,StdAge,StudyType
0,NCT00000152,Unknown status,0,0,0,0,0,"[D000012162, D000012164, D000005128]","[BC11, All]","[M9843, M13580, M13582, M6854]",...,"[M3129, M4174, M19779, M2798, M2799, M2613, M1...","[D000001241, D000019207, D000002338]","[Drug, Drug]",0,0,NIH,0,0,"[Adult, Older Adult]",Interventional
1,NCT00000157,Terminated,0,0,0,0,0,"[D000007905, D000005128]","[BC11, All]","[M4221, M9500, M6854]",...,"[M3129, M2798, M2799, M2613, M19370, M19188, M...",D000001241,Drug,0,0,NIH,0,0,"[Adult, Older Adult]",Interventional
2,NCT00000172,Completed,0,0,0,0,0,"[D000003704, D000001927, D000002493, D00000942...","[BC10, BXM, All, Rare]","[M2466, M5487, M3786, M4325, M21586, M20142, M...",...,"[M19342, M4623, M7403, M19088]",D000005702,Drug,0,0,INDUSTRY,0,0,"[Child, Adult, Older Adult]",Interventional
3,NCT00000173,Completed,0,0,0,0,OTHER,"[D000003704, D000001927, D000002493, D00000942...","[BXM, All, BC10, BC23, Rare]","[M28292, M2466, M19143, M5487, M3786, M4325, M...",...,"[M16141, M16136, M2873, M1721, M21556, M21553,...","[D000014810, D000077265]","[Drug, Drug]",0,0,NIH,0,0,"[Adult, Older Adult]",Interventional
4,NCT00000174,Completed,0,0,0,0,0,"[D000003704, D000001927, D000002493, D00000942...","[BXM, All, BC10, Rare]","[M28292, M2466, M4884, M5487, M3786, M4325, M2...",...,"[M317, M19342, M4623, M19088, M19357, M20453]",D000068836,Drug,0,0,INDUSTRY,0,0,"[Adult, Older Adult]",Interventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NCT04608188,Recruiting,Behavioral: Summer Camp,"[Experimental, No Intervention]",0,0,0,"[D000001836, D000001835]","[BC23, All, BC18]","[M16684, M11284, M3696, M3697]",...,0,0,Behavioral,No,No,OTHER,0,0,Child,Interventional
9996,NCT04608773,Not yet recruiting,[Other: Remineralizing Extreme Dry Mouth Spray...,"[Other, Other]",0,0,UNKNOWN,"[D000009371, D000009369, D000012466, D00000905...","[BC04, All, BC07]","[M7931, M16307, M13868, M10602, M10600]",...,"[M2358, M5878]",0,"[Other, Device]",Yes,No,OTHER,0,0,"[Adult, Older Adult]",Interventional
9997,NCT04609085,Enrolling by invitation,0,0,0,0,0,"[D000020969, D000010335]","[BC23, All, BC02]","[M23105, M16105, M21284]",...,"[M8767, M2806]",0,0,No,No,NIH,0,Non-Probability Sample,"[Child, Adult, Older Adult]",Observational
9998,NCT04609514,Not yet recruiting,"[Device: Learn to Quit-HIV, Drug: Nicotine pat...","[Experimental, Active Comparator]",0,0,NIH,"[D000019966, D000064419, D000001523]","[BC25, BXM, All, BC02, BC20]","[M15368, M16833, M2103, M20421, M28889, M3396,...",...,"[M11061, M2610, M19380, M19342, M19088]",D000009538,"[Device, Behavioral, Drug, Behavioral]",Yes,No,OTHER,0,0,"[Adult, Older Adult]",Interventional


In [9]:
# attach the Phases to the new table
df = pd.concat([phase_crosstb, df], axis=1, sort=False)
df = df.drop(df.columns[6], axis=1)
df

,Phase_1,Phase_2,Phase_3,Phase_4,n/a,Status,ArmGroupInterventionName,ArmGroupType,BaselineMeasureDispersionType,BaselineMeasureParamType,...,InterventionBrowseLeafId,InterventionMeshId,InterventionType,IsFDARegulatedDevice,IsFDARegulatedDrug,LeadSponsorClass,PatientRegistry,SamplingMethod,StdAge,StudyType
0,0,0,1,0,0,Unknown status,0,0,0,0,...,"[M3129, M4174, M19779, M2798, M2799, M2613, M1...","[D000001241, D000019207, D000002338]","[Drug, Drug]",0,0,NIH,0,0,"[Adult, Older Adult]",Interventional
1,0,0,1,0,0,Terminated,0,0,0,0,...,"[M3129, M2798, M2799, M2613, M19370, M19188, M...",D000001241,Drug,0,0,NIH,0,0,"[Adult, Older Adult]",Interventional
2,0,0,1,0,0,Completed,0,0,0,0,...,"[M19342, M4623, M7403, M19088]",D000005702,Drug,0,0,INDUSTRY,0,0,"[Child, Adult, Older Adult]",Interventional
3,0,0,1,0,0,Completed,0,0,0,0,...,"[M16141, M16136, M2873, M1721, M21556, M21553,...","[D000014810, D000077265]","[Drug, Drug]",0,0,NIH,0,0,"[Adult, Older Adult]",Interventional
4,0,0,1,0,0,Completed,0,0,0,0,...,"[M317, M19342, M4623, M19088, M19357, M20453]",D000068836,Drug,0,0,INDUSTRY,0,0,"[Adult, Older Adult]",Interventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,1,Recruiting,Behavioral: Summer Camp,"[Experimental, No Intervention]",0,0,...,0,0,Behavioral,No,No,OTHER,0,0,Child,Interventional
9996,0,0,0,0,1,Not yet recruiting,[Other: Remineralizing Extreme Dry Mouth Spray...,"[Other, Other]",0,0,...,"[M2358, M5878]",0,"[Other, Device]",Yes,No,OTHER,0,0,"[Adult, Older Adult]",Interventional
9997,0,0,0,0,1,Enrolling by invitation,0,0,0,0,...,"[M8767, M2806]",0,0,No,No,NIH,0,Non-Probability Sample,"[Child, Adult, Older Adult]",Observational
9998,0,0,0,0,1,Not yet recruiting,"[Device: Learn to Quit-HIV, Drug: Nicotine pat...","[Experimental, Active Comparator]",0,0,...,"[M11061, M2610, M19380, M19342, M19088]",D000009538,"[Device, Behavioral, Drug, Behavioral]",Yes,No,OTHER,0,0,"[Adult, Older Adult]",Interventional


In [ ]:
tb1= explodeTable(df, 'ArmGroupInterventionName')
tb1['feature'] = tb1['ArmGroupInterventionName'].str.split(' ').str[0]
tb1 = tb1.drop(['ArmGroupInterventionName'], axis=1)
tb1 = tb1.rename(columns={'feature':'ArmGroupInterventionName'})
tb1.to_csv('SampleFeatureTables/ArmGroupInterventionName.csv')
tb1

In [ ]:
tb2= explodeTable(df, 'ArmGroupType')
tb2.to_csv('SampleFeatureTables/ArmGroupType.csv')
tb2

In [ ]:
tb3= explodeTable(df, 'BaselineMeasureDispersionType')
tb3.to_csv('SampleFeatureTables/BaselineMeasureDispersionType.csv')
tb3

In [ ]:
tb4= explodeTable(df, 'BaselineMeasureParamType')
tb4.to_csv('SampleFeatureTables/BaselineMeasureParamType.csv')
tb4

In [ ]:
tb5= explodeTable(df, 'CollaboratorClass')
tb5.to_csv('SampleFeatureTables/CollaboratorClass.csv')
tb5

In [ ]:
#some problem please look again
#tb6= explodeTable(df_from_json, 'ConditionAncestorId')
#tb6.to_csv('SampleFeatureTables/ConditionAncestorId.csv')
#tb6

In [ ]:
tb7= explodeTable(df, 'ConditionBrowseBranchAbbrev')
tb7.to_csv('SampleFeatureTables/ConditionBrowseBranchAbbrev.csv')
tb7

In [ ]:
tb8= explodeTable(df, 'ConditionBrowseLeafId')
tb8.to_csv('SampleFeatureTables/ConditionBrowseLeafId.csv')
tb8

In [ ]:
tb9= explodeTable(df, 'ConditionMeshId')
tb9.to_csv('SampleFeatureTables/ConditionMeshId.csv')
tb9

In [ ]:
tb10= explodeTable(df, 'DesignAllocation')
tb10.to_csv('SampleFeatureTables/DesignAllocation.csv')
tb10

In [ ]:
tb11= explodeTable(df, 'DesignMasking')
tb11.to_csv('SampleFeatureTables/DesignMasking.csv')
tb11

In [ ]:
tb12= explodeTable(df, 'DesignPrimaryPurpose')
tb12.to_csv('SampleFeatureTables/DesignPrimaryPurpose.csv')
tb12

In [ ]:
tb13= explodeTable(df, 'DesignTimePerspective')
tb13.to_csv('SampleFeatureTables/DesignTimePerspective.csv')
tb13

In [ ]:
tb14= explodeTable(df, 'EnrollmentCount')
tb14.to_csv('SampleFeatureTables/EnrollmentCount.csv')
tb14

In [ ]:
tb15= explodeTable(df, 'HealthyVolunteers')
tb15.to_csv('SampleFeatureTables/HealthyVolunteers.csv')
tb15

In [ ]:
tb16= explodeTable(df, 'InterventionAncestorId')
tb16.to_csv('SampleFeatureTables/InterventionAncestorId.csv')
tb16

In [ ]:
tb17= explodeTable(df, 'InterventionBrowseBranchAbbrev')
tb17.to_csv('SampleFeatureTables/InterventionBrowseBranchAbbrev.csv')
tb17

In [ ]:
tb18= explodeTable(df, 'InterventionBrowseLeafId')
tb18.to_csv('SampleFeatureTables/InterventionBrowseLeafId.csv')
tb18

In [ ]:
tb19= explodeTable(df, 'InterventionMeshId')
tb19.to_csv('SampleFeatureTables/InterventionMeshId.csv')
tb19

In [ ]:
tb20= explodeTable(df, 'InterventionType')
tb20.to_csv('SampleFeatureTables/InterventionType.csv')
tb20

In [ ]:
tb20= explodeTable(df, 'InterventionType')
tb20.to_csv('SampleFeatureTables/InterventionType.csv')
tb20

In [ ]:
tb21= explodeTable(df, 'IsFDARegulatedDevice')
tb21.to_csv('SampleFeatureTables/IsFDARegulatedDevice.csv')
tb21

In [ ]:
tb22= explodeTable(df, 'IsFDARegulatedDrug')
tb22.to_csv('SampleFeatureTables/IsFDARegulatedDrug.csv')
tb22

In [ ]:
tb23= explodeTable(df, 'LeadSponsorClass')
tb23.to_csv('SampleFeatureTables/LeadSponsorClass.csv')
tb23

In [ ]:
tb25= explodeTable(df, 'PatientRegistry')
tb25.to_csv('SampleFeatureTables/PatientRegistry.csv')
tb25

In [ ]:
#run into prolem need to check later
#tb27= explodeTable(df_from_json, 'SamplingMethod')
#tb27.to_csv('SampleFeatureTables/SamplingMethod.csv')
#tb27

In [ ]:
tb28= explodeTable(df, 'StdAge')
tb28.to_csv('SampleFeatureTables/StdAge.csv')
tb28

In [ ]:
tb29= explodeTable(df, 'StudyType')
tb29.to_csv('SampleFeatureTables/StudyType.csv')
tb29